# To Grant or Not to Grant: Deciding on Compensation Benefits 

### Machine Learning Group Project 2024/2025 - Group 52


__Members:__
- Diogo Rodrigues _(20240512@novaims.unl.pt)_
- Alexandre Gonçalves _(20240738@novaims.unl.pt)_
- Pedro Costa _(20222121@novaims.unl.pt)_
- Santiago Taylor _(20240542@novaims.unl.pt)_
- Lena Hermann _(20241446@novaims.unl.pt)_

## Introduction
- __Motivation__: The New York Workers’ Compensation Board (WCB) handles a vast number of workplace injury claims, which require timely and accurate decisions. Currently, manually reviewing these claims is both resource-intensive and time-consuming. An automated model could significantly expedite the decision-making process, improving efficiency and consistency in awarding compensation.
- __Dataset__: The dataset contains claims assembled by the WCB from the beginning of 2020 to the end of 2022, which serves as the training data. The test data comprises claims from January 2023 onward. Each claim includes various attributes such as injury details, claimant information, and associated codes that characterize the nature, cause, and affected body part of the injury. This comprehensive dataset enables the development of predictive models based on historical decisions.
- __Task and Goal__: The primary objective of this project is to develop and evaluate machine learning models that can accurately classify the WCB’s final decision on the type of injury for each claim (i.e., the "Claim Injury Type"). This involves creating a multiclass classification model to predict injury types based on the provided features, optimizing model performance, and assessing the importance of different features. The ultimate goal is to produce a model that can generalize well to unseen data, assisting the WCB in making faster, data-driven decisions on new claims.

| **ATTRIBUTE**                        | **DESCRIPTION**                                                                                                         |
|--------------------------------------|-------------------------------------------------------------------------------------------------------------------------|
| Accident Date                        | Injury date of the claim.                                                                                               |
| Age at Injury                        | Age of injured worker when the injury occurred.                                                                         |
| Alternative Dispute Resolution       | Adjudication processes external to the Board.                                                                           |
| Assembly Date                        | The date the claim was first assembled.                                                                                 |
| Attorney/Representative              | Is the claim being represented by an Attorney?                                                                          |
| Average Weekly Wage                  | The wage used to calculate workers’ compensation, disability, or Paid Leave wage replacement benefits.                  |
| Birth Year                           | The reported year of birth of the injured worker.                                                                       |
| C-2 Date                             | Date of receipt of the Employer's Report of Work-Related Injury/Illness or equivalent (formerly Form C-2).              |
| C-3 Date                             | Date Form C-3 (Employee Claim Form) was received.                                                                       |
| Carrier Name                         | Name of primary insurance provider responsible for providing workers’ compensation coverage to the injured worker's employer. |
| Carrier Type                         | Type of primary insurance provider responsible for providing workers’ compensation coverage.                            |
| Claim Identifier                     | Unique identifier for each claim, assigned by WCB.                                                                      |
| County of Injury                     | Name of the New York County where the injury occurred.                                                                  |
| COVID-19 Indicator                   | Indication that the claim may be associated with COVID-19.                                                              |
| District Name                        | Name of the WCB district office that oversees claims for that region or area of the state.                              |
| First Hearing Date                   | Date the first hearing was held on a claim at a WCB hearing location. A blank date means the claim has not yet had a hearing held. |
| Gender                               | The reported gender of the injured worker.                                                                              |
| IME-4 Count                          | Number of IME-4 forms received per claim. The IME-4 form is the “Independent Examiner’s Report of Independent Medical Examination” form. |
| Industry Code                        | NAICS code and descriptions are available at https://www.naics.com/search-naics-codes-by-industry/.                     |
| Industry Code Description            | 2-digit NAICS industry code description used to classify businesses according to their economic activity.               |
| Medical Fee Region                   | Approximate region where the injured worker would receive medical service.                                               |
| OIICS Nature of Injury Description   | The OIICS nature of injury codes & descriptions are available at https://www.bls.gov/iif/oiics_manual_2007.pdf.         |
| WCIO Cause of Injury Code            | The WCIO cause of injury codes & descriptions are available at https://www.wcio.org/Active%20PNC/WCIO_Cause_Table.pdf.  |
| WCIO Cause of Injury Description     | See description of field above.                                                                                         |
| WCIO Nature of Injury Code           | The WCIO nature of injury are available at https://www.wcio.org/Active%20PNC/WCIO_Nature_Table.pdf.                     |
| WCIO Nature of Injury Description    | See description of field above.                                                                                         |
| WCIO Part Of Body Code               | The WCIO part of body codes & descriptions are available at https://www.wcio.org/Active%20PNC/WCIO_Part_Table.pdf.      |
| WCIO Part Of Body Description        | See description of field above.                                                                                         |
| Zip Code                             | The reported ZIP code of the injured worker’s home address.                                                             |
| Agreement Reached                    | Binary variable: Yes if there is an agreement without the involvement of the WCB -> unknown at the start of a claim.    |
| WCB Decision                         | Multiclass variable: Decision of the WCB relative to the claim: “Accident” means that claim refers to workplace accident, “Occupational Disease” means illness from the workplace. -> requires WCB deliberation so it is unknown at start of claim. |
| Claim Injury Type                    | Main target variable: Deliberation of the WCB relative to benefits awarded to the claim. Numbering indicates severity.   |


## Table of Content
<a id='top'></a>
[1. Set-Up and Data Loading](#dataload) <br>
&nbsp;&nbsp;&nbsp;[1.1 Import the Needed Libraries](#import_libaries) <br>
&nbsp;&nbsp;&nbsp;[1.2 Read the Data and First Overview](#read_data) <br>

## 1. Set-Up and Data Loading <a id='dataload'></a>
[to the top](#top)

### 1.1 Import the Needed Libraries <a id='import_libaries'></a>

In [2]:
import numpy as np #to efficiently compute functions
import pandas as pd #to work with data frames
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import zipfile #to open zipfile
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 60)
from scipy.stats import mode

# data partition
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

#filter methods
# spearman 
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

# embedded methods
from sklearn.linear_model import LassoCV

# encoding and scaling
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler

# model selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# models
from sklearn.ensemble import RandomForestClassifier

# model assessment
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### 1.2 Read the Data and First Overview <a id='read_data'></a>
[to the top](#top)

In [3]:
train_zip_path = 'train_data.csv.zip'
test_zip_path = 'test_data.csv.zip'

# open csv file from zip
def read_zipped_csv(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as z:
        csv_filename = z.namelist()[0]
        
        with z.open(csv_filename) as f:
            return pd.read_csv(f)

train_df = read_zipped_csv(train_zip_path)
test_df = read_zipped_csv(test_zip_path)

In [4]:
train_df.head()

Accident Date  Age at Injury Alternative Dispute Resolution Assembly Date  \
0    2019-12-30           31.0                              N    2020-01-01   
1    2019-08-30           46.0                              N    2020-01-01   
2    2019-12-06           40.0                              N    2020-01-01   
3           NaN            NaN                            NaN    2020-01-01   
4    2019-12-30           61.0                              N    2020-01-01   

  Attorney/Representative  Average Weekly Wage  Birth Year    C-2 Date  \
0                       N                 0.00      1988.0  2019-12-31   
1                       Y              1745.93      1973.0  2020-01-01   
2                       N              1434.80      1979.0  2020-01-01   
3                     NaN                  NaN         NaN         NaN   
4                       N                  NaN      1958.0  2019-12-31   

     C-3 Date                  Carrier Name Carrier Type  Claim Identifier  \
0         NaN    NEW HAMPSHIRE INSURANCE CO  1A. PRIVATE           5393875   
1  2020-01-14  ZURICH AMERICAN INSURANCE CO  1A. PRIVATE           5393091   
2         NaN     INDEMNITY INSURANCE CO OF  1A. PRIVATE           5393889   
3         NaN                           NaN          NaN         957648180   
4         NaN          STATE INSURANCE FUND      2A. SIF           5393887   

  Claim Injury Type County of Injury COVID-19 Indicator District Name  \
0       2. NON-COMP     ST. LAWRENCE                  N      SYRACUSE   
1      4. TEMPORARY          WYOMING                  N     ROCHESTER   
2      4. TEMPORARY           ORANGE                  N        ALBANY   
3               NaN              NaN                NaN           NaN   
4       2. NON-COMP         DUTCHESS                  N        ALBANY   

  First Hearing Date Gender  IME-4 Count  Industry Code  \
0                NaN      M          NaN           44.0   
1         2020-02-21      F          4.0           23.0   
2                NaN      M          NaN           56.0   
3                NaN    NaN          NaN            NaN   
4                NaN      M          NaN           62.0   

                           Industry Code Description Medical Fee Region  \
0                                       RETAIL TRADE                  I   
1                                       CONSTRUCTION                  I   
2  ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...                 II   
3                                                NaN                NaN   
4                  HEALTH CARE AND SOCIAL ASSISTANCE                 II   

   OIICS Nature of Injury Description  WCIO Cause of Injury Code  \
0                                 NaN                       27.0   
1                                 NaN                       97.0   
2                                 NaN                       79.0   
3                                 NaN                        NaN   
4                                 NaN                       16.0   

  WCIO Cause of Injury Description  WCIO Nature of Injury Code  \
0     FROM LIQUID OR GREASE SPILLS                        10.0   
1                REPETITIVE MOTION                        49.0   
2   OBJECT BEING LIFTED OR HANDLED                         7.0   
3                              NaN                         NaN   
4  HAND TOOL, UTENSIL; NOT POWERED                        43.0   

  WCIO Nature of Injury Description  WCIO Part Of Body Code  \
0                         CONTUSION                    62.0   
1                    SPRAIN OR TEAR                    38.0   
2                        CONCUSSION                    10.0   
3                               NaN                     NaN   
4                          PUNCTURE                    36.0   

  WCIO Part Of Body Description Zip Code  Agreement Reached      WCB Decision  \
0                      BUTTOCKS    13662                0.0  Not Work Related   
1                   S

In [5]:
test_df.head()

Accident Date  Age at Injury Alternative Dispute Resolution Assembly Date  \
0    2022-12-24             19                              N    2023-01-02   
1    2022-11-20             19                              N    2023-01-02   
2    2022-12-26             59                              N    2023-01-02   
3    2022-12-28             55                              N    2023-01-02   
4    2022-12-20             25                              N    2023-01-02   

  Attorney/Representative  Average Weekly Wage  Birth Year    C-2 Date  \
0                       N                  NaN      2003.0  2023-01-02   
1                       N                  NaN      2003.0  2023-01-02   
2                       N                  0.0      1963.0  2022-12-31   
3                       N                  0.0         0.0  2023-01-02   
4                       N                  0.0      1997.0  2022-12-31   

  C-3 Date                 Carrier Name Carrier Type  Claim Identifier  \
0      NaN    INDEMNITY INSURANCE CO OF  1A. PRIVATE           6165911   
1      NaN      A I U INSURANCE COMPANY  1A. PRIVATE           6166141   
2      NaN    AMGUARD INSURANCE COMPANY  1A. PRIVATE           6165907   
3      NaN  INDEMNITY INS. OF N AMERICA  1A. PRIVATE           6166047   
4      NaN   NEW HAMPSHIRE INSURANCE CO  1A. PRIVATE           6166102   

  County of Injury COVID-19 Indicator District Name First Hearing Date Gender  \
0            BRONX                  N           NYC                NaN      M   
1           QUEENS                  N           NYC                NaN      F   
2      WESTCHESTER                  N           NYC                NaN      F   
3           QUEENS                  N           NYC                NaN      F   
4            KINGS                  N           NYC                NaN      M   

   IME-4 Count  Industry Code  \
0          NaN           48.0   
1          NaN           45.0   
2          NaN           56.0   
3          NaN           48.0   
4          NaN           55.0   

                           Industry Code Description Medical Fee Region  \
0                     TRANSPORTATION AND WAREHOUSING                 IV   
1                                       RETAIL TRADE                 IV   
2  ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...                III   
3                     TRANSPORTATION AND WAREHOUSING                 IV   
4            MANAGEMENT OF COMPANIES AND ENTERPRISES                 IV   

   OIICS Nature of Injury Description  WCIO Cause of Injury Code  \
0                                 NaN                       31.0   
1                                 NaN                       75.0   
2                                 NaN                       68.0   
3                                 NaN                       25.0   
4                                 NaN                       79.0   

   WCIO Cause of Injury Description  WCIO Nature of Injury Code  \
0           FALL, SLIP OR TRIP, NOC                        10.0   
1          FALLING OR FLYING OBJECT                        10.0   
2                 STATIONARY OBJECT                        49.0   
3  FROM DIFFERENT LEVEL (ELEVATION)                        10.0   
4    OBJECT BEING LIFTED OR HANDLED                        40.0   

  WCIO Nature of Injury Description  WCIO Part Of Body Code  \
0                         CONTUSION                    54.0   
1                         CONTUSION                    10.0   
2                    SPRAIN OR TEAR                    62.0   
3                         CONTUSION                    53.0   
4                        LACERATION                    37.0   

  WCIO Part Of Body Description Zip Code  Number of Dependents  
0                     LOWER LEG    10466                     1  
1          MULTIPLE HEAD INJURY    11691                     1  
2                      BUTTOCKS    10604                     0  
3                          KNEE    11411         

## 2. Data Understanding, Consistency and Feature Engineering <a id='dataunderstand'></a>
[to the top](#top)

### Missing Values

In [6]:
train_df.isnull().sum()

Accident Date                          23134
Age at Injury                          19445
Alternative Dispute Resolution         19445
Assembly Date                              0
Attorney/Representative                19445
Average Weekly Wage                    48096
Birth Year                             48523
C-2 Date                               34005
C-3 Date                              406226
Carrier Name                           19445
Carrier Type                           19445
Claim Identifier                           0
Claim Injury Type                      19445
County of Injury                       19445
COVID-19 Indicator                     19445
District Name                          19445
First Hearing Date                    442673
Gender                                 19445
IME-4 Count                           460668
Industry Code                          29403
Industry Code Description              29403
Medical Fee Region                     19445
OIICS Natu

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387975 entries, 0 to 387974
Data columns (total 30 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       385531 non-null  object 
 1   Age at Injury                       387975 non-null  int64  
 2   Alternative Dispute Resolution      387975 non-null  object 
 3   Assembly Date                       387975 non-null  object 
 4   Attorney/Representative             387975 non-null  object 
 5   Average Weekly Wage                 368771 non-null  float64
 6   Birth Year                          368505 non-null  float64
 7   C-2 Date                            378841 non-null  object 
 8   C-3 Date                            85216 non-null   object 
 9   Carrier Name                        387975 non-null  object 
 10  Carrier Type                        387975 non-null  object 
 11  Claim Identifier          

### Duplicates

In [8]:
# checking for duplicates
train_df.duplicated().sum()

np.int64(0)

In [9]:
test_df.duplicated().sum()

np.int64(0)

In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593471 entries, 0 to 593470
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       570337 non-null  object 
 1   Age at Injury                       574026 non-null  float64
 2   Alternative Dispute Resolution      574026 non-null  object 
 3   Assembly Date                       593471 non-null  object 
 4   Attorney/Representative             574026 non-null  object 
 5   Average Weekly Wage                 545375 non-null  float64
 6   Birth Year                          544948 non-null  float64
 7   C-2 Date                            559466 non-null  object 
 8   C-3 Date                            187245 non-null  object 
 9   Carrier Name                        574026 non-null  object 
 10  Carrier Type                        574026 non-null  object 
 11  Claim Identifier          

## Check Data Consistency

### Codes Analysis

In [11]:
def analyze_code_description_relationship(data, code_column, description_column):
    # Step 1: Count unique descriptions per code
    code_to_description = data.groupby(code_column)[description_column].nunique()
    # Check if each code maps to a single unique description
    if code_to_description.max() == 1:
        print(f"\nEach {code_column} has only one unique {description_column}.")
    else:
        print(f"\nSome {code_column}s have multiple {description_column}s associated with them.")

    # Step 2: Count unique codes per description
    description_to_code = data.groupby(description_column)[code_column].nunique()
    # Check if each description maps to a single unique code
    if description_to_code.max() == 1:
        print(f"\nEach {description_column} has only one unique {code_column}.")
    else:
        print(f"\nSome {description_column}s have multiple {code_column}s associated with them.")

    # Step 3: Validate if there is a 1:1 relationship
    is_one_to_one = (code_to_description.max() == 1) and (description_to_code.max() == 1)
    if is_one_to_one:
        print(f"\n{code_column} and {description_column} have a 1:1 relationship.")
    else:
        print(f"\nThere is no 1:1 relationship between {code_column} and {description_column}.")

    # Step 4: Find descriptions with multiple unique associated codes
    description_to_unique_codes = data.groupby(description_column)[code_column].apply(lambda x: list(set(x)))
    descriptions_with_multiple_unique_codes = description_to_unique_codes[description_to_unique_codes.apply(len) > 1]
    
    print(f"\nDescriptions with multiple unique associated {code_column}s:")
    print(descriptions_with_multiple_unique_codes)

__Industry Code / Industry Code Description__

In [12]:
analyze_code_description_relationship(train_df, 'Industry Code', 'Industry Code Description')


Each Industry Code has only one unique Industry Code Description.

Some Industry Code Descriptions have multiple Industry Codes associated with them.

There is no 1:1 relationship between Industry Code and Industry Code Description.

Descriptions with multiple unique associated Industry Codes:
Industry Code Description
MANUFACTURING                     [32.0, 33.0, 31.0]
RETAIL TRADE                            [44.0, 45.0]
TRANSPORTATION AND WAREHOUSING          [48.0, 49.0]
Name: Industry Code, dtype: object


__WCIO Cause of Injury Code / WCIO Cause of Injury Description__

In [13]:
analyze_code_description_relationship(train_df, 'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description')


Each WCIO Cause of Injury Code has only one unique WCIO Cause of Injury Description.

Some WCIO Cause of Injury Descriptions have multiple WCIO Cause of Injury Codes associated with them.

There is no 1:1 relationship between WCIO Cause of Injury Code and WCIO Cause of Injury Description.

Descriptions with multiple unique associated WCIO Cause of Injury Codes:
WCIO Cause of Injury Description
OBJECT BEING LIFTED OR HANDLED    [17.0, 66.0, 79.0]
REPETITIVE MOTION                       [97.0, 94.0]
Name: WCIO Cause of Injury Code, dtype: object


__WCIO Nature of Injury Code / WCIO Nature of Injury Description__

In [14]:
analyze_code_description_relationship(train_df, 'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description')


Each WCIO Nature of Injury Code has only one unique WCIO Nature of Injury Description.

Each WCIO Nature of Injury Description has only one unique WCIO Nature of Injury Code.

WCIO Nature of Injury Code and WCIO Nature of Injury Description have a 1:1 relationship.

Descriptions with multiple unique associated WCIO Nature of Injury Codes:
Series([], Name: WCIO Nature of Injury Code, dtype: object)


__WCIO Part Of Body Code / WCIO Part Of Body Description__

In [15]:
analyze_code_description_relationship(train_df, 'WCIO Part Of Body Code', 'WCIO Part Of Body Description')


Each WCIO Part Of Body Code has only one unique WCIO Part Of Body Description.

Some WCIO Part Of Body Descriptions have multiple WCIO Part Of Body Codes associated with them.

There is no 1:1 relationship between WCIO Part Of Body Code and WCIO Part Of Body Description.

Descriptions with multiple unique associated WCIO Part Of Body Codes:
WCIO Part Of Body Description
DISC           [43.0, 22.0]
SOFT TISSUE    [25.0, 18.0]
SPINAL CORD    [47.0, 23.0]
Name: WCIO Part Of Body Code, dtype: object


# Feature Engineering

### Mapping for WCIO features

__WCIO Cause of Injury Description__

In [16]:
# Mapping dictionary
injury_mapping = {
    # Falls and Slips
    'FALL, SLIP OR TRIP, NOC': 'Falls and Slips',
    'FROM LIQUID OR GREASE SPILLS': 'Falls and Slips',
    'ON ICE OR SNOW': 'Falls and Slips',
    'ON SAME LEVEL': 'Falls and Slips',
    'ON STAIRS': 'Falls and Slips',
    'SLIP, OR TRIP, DID NOT FALL': 'Falls and Slips',
    
    # Lifting and Strain
    'OBJECT BEING LIFTED OR HANDLED': 'Lifting and Strain',
    'LIFTING': 'Lifting and Strain',
    'TWISTING': 'Lifting and Strain',
    'PUSHING OR PULLING': 'Lifting and Strain',
    'STRAIN OR INJURY BY, NOC': 'Lifting and Strain',
    'REACHING': 'Lifting and Strain',
    'HOLDING OR CARRYING': 'Lifting and Strain',
    
    # Collisions and Motor Vehicle Accidents
    'COLLISION OR SIDESWIPE WITH ANOTHER VEHICLE': 'Collisions and Motor Vehicle Accidents',
    'MOTOR VEHICLE, NOC': 'Collisions and Motor Vehicle Accidents',
    'MOTOR VEHICLE': 'Collisions and Motor Vehicle Accidents',
    'COLLISION WITH A FIXED OBJECT': 'Collisions and Motor Vehicle Accidents',
    'CRASH OF RAIL VEHICLE': 'Collisions and Motor Vehicle Accidents',
    'CRASH OF WATER VEHICLE': 'Collisions and Motor Vehicle Accidents',
    'CRASH OF AIRPLANE': 'Collisions and Motor Vehicle Accidents',
    'VEHICLE UPSET': 'Collisions and Motor Vehicle Accidents',
    
    # Struck By Object or Person
    'STRUCK OR INJURED, NOC': 'Struck By Object or Person',
    'FALLING OR FLYING OBJECT': 'Struck By Object or Person',
    'STEPPING ON SHARP OBJECT': 'Struck By Object or Person',
    'FELLOW WORKER, PATIENT OR OTHER PERSON': 'Struck By Object or Person',
    'ANIMAL OR INSECT': 'Struck By Object or Person',
    'WIELDING OR THROWING': 'Struck By Object or Person',
    
    # Exposure to Substances and Environment
    'CHEMICALS': 'Exposure to Substances and Environment',
    'DUST, GASES, FUMES OR VAPORS': 'Exposure to Substances and Environment',
    'ABSORPTION, INGESTION OR INHALATION, NOC': 'Exposure to Substances and Environment',
    'STEAM OR HOT FLUIDS': 'Exposure to Substances and Environment',
    'COLD OBJECTS OR SUBSTANCES': 'Exposure to Substances and Environment',
    'TEMPERATURE EXTREMES': 'Exposure to Substances and Environment',
    'CONTINUAL NOISE': 'Exposure to Substances and Environment',
    'MOLD': 'Exposure to Substances and Environment',
    'ABNORMAL AIR PRESSURE': 'Exposure to Substances and Environment',
    'RADIATION': 'Exposure to Substances and Environment',
    
    # Tools, Machinery, and Objects
    'HAND TOOL, UTENSIL; NOT POWERED': 'Tools, Machinery, and Objects',
    'POWERED HAND TOOL, APPLIANCE': 'Tools, Machinery, and Objects',
    'USING TOOL OR MACHINERY': 'Tools, Machinery, and Objects',
    'MOVING PARTS OF MACHINE': 'Tools, Machinery, and Objects',
    'MACHINE OR MACHINERY': 'Tools, Machinery, and Objects',
    'MOVING PART OF MACHINE': 'Tools, Machinery, and Objects',
    'OBJECT HANDLED BY OTHERS': 'Tools, Machinery, and Objects',
    'BROKEN GLASS': 'Tools, Machinery, and Objects',
    
    # Explosions and Fire
    'FIRE OR FLAME': 'Explosions and Fire',
    'EXPLOSION OR FLARE BACK': 'Explosions and Fire',
    
    # Natural Disasters and Uncommon Events
    'NATURAL DISASTERS': 'Natural Disasters and Uncommon Events',
    'TERRORISM': 'Natural Disasters and Uncommon Events',
    'PANDEMIC': 'Natural Disasters and Uncommon Events',
}

__WCIO Nature of Injury Description__

In [17]:
# Mapping dictionary
injury_nature_mapping = {
    # Physical Trauma
    'CONTUSION': 'Physical Trauma',
    'SPRAIN OR TEAR': 'Physical Trauma',
    'CONCUSSION': 'Physical Trauma',
    'PUNCTURE': 'Physical Trauma',
    'LACERATION': 'Physical Trauma',
    'FRACTURE': 'Physical Trauma',
    'CRUSHING': 'Physical Trauma',
    'DISLOCATION': 'Physical Trauma',
    'AMPUTATION': 'Physical Trauma',
    'SEVERANCE': 'Physical Trauma',
    
    # Cumulative or Repetitive Injuries
    'CARPAL TUNNEL SYNDROME': 'Cumulative or Repetitive Injuries',
    'INFLAMMATION': 'Cumulative or Repetitive Injuries',
    'ALL OTHER CUMULATIVE INJURY, NOC': 'Cumulative or Repetitive Injuries',
    'ALL OTHER OCCUPATIONAL DISEASE INJURY, NOC': 'Cumulative or Repetitive Injuries',
    'ALL OTHER SPECIFIC INJURIES, NOC': 'Cumulative or Repetitive Injuries',
    
    # Burns and Temperature-Related Injuries
    'BURN': 'Burns and Temperature-Related Injuries',
    'HEAT PROSTRATION': 'Burns and Temperature-Related Injuries',
    'FREEZING': 'Burns and Temperature-Related Injuries',
    
    # Respiratory Issues
    'RESPIRATORY DISORDERS': 'Respiratory Issues',
    'ASBESTOSIS': 'Respiratory Issues',
    'DUST DISEASE, NOC': 'Respiratory Issues',
    'BLACK LUNG': 'Respiratory Issues',
    'SILICOSIS': 'Respiratory Issues',
    'BYSSINOSIS': 'Respiratory Issues',
    
    # Poisoning and Chemical Exposure
    'POISONING - GENERAL (NOT OD OR CUMULATIVE': 'Poisoning and Chemical Exposure',
    'POISONING - CHEMICAL, (OTHER THAN METALS)': 'Poisoning and Chemical Exposure',
    'POISONING - METAL': 'Poisoning and Chemical Exposure',
    
    # Infections and Diseases
    'INFECTION': 'Infections and Diseases',
    'COVID-19': 'Infections and Diseases',
    'CONTAGIOUS DISEASE': 'Infections and Diseases',
    'AIDS': 'Infections and Diseases',
    'HEPATITIS C': 'Infections and Diseases',
    'CANCER': 'Infections and Diseases',
    
    # Psychological and Mental Health
    'MENTAL STRESS': 'Psychological and Mental Health',
    'MENTAL DISORDER': 'Psychological and Mental Health',
    'NO PHYSICAL INJURY': 'Psychological and Mental Health',
    'MULTIPLE INJURIES INCLUDING BOTH PHYSICAL AND PSYCHOLOGICAL': 'Psychological and Mental Health',
    
    # Circulatory and Heart-Related Issues
    'MYOCARDIAL INFARCTION': 'Circulatory and Heart-Related Issues',
    'ANGINA PECTORIS': 'Circulatory and Heart-Related Issues',
    'VASCULAR': 'Circulatory and Heart-Related Issues',
    
    # Sensory Loss
    'LOSS OF HEARING': 'Sensory Loss',
    'HEARING LOSS OR IMPAIRMENT': 'Sensory Loss',
    'VISION LOSS': 'Sensory Loss',
    'ENUCLEATION': 'Sensory Loss',
    
    # Radiation and Light Exposure
    'RADIATION': 'Radiation and Light Exposure',
    'VDT - RELATED DISEASES': 'Radiation and Light Exposure',
    
    # Miscellaneous (catch-all for remaining cases)
    'ASPHYXIATION': 'Miscellaneous Conditions',
    'SYNCOPE': 'Miscellaneous Conditions',
    'DERMATITIS': 'Miscellaneous Conditions',
    'ADVERSE REACTION TO A VACCINATION OR INOCULATION': 'Miscellaneous Conditions'
}

__WCIO Part of Body Description__

In [18]:
# Mapping dictionary
body_part_mapping = {
    # Head and Face
    'SKULL': 'Head and Face', 'FACIAL BONES': 'Head and Face', 'EYE(S)': 'Head and Face',
    'MOUTH': 'Head and Face', 'NOSE': 'Head and Face', 'EAR(S)': 'Head and Face', 
    'BRAIN': 'Head and Face', 'TEETH': 'Head and Face',

    # Neck and Throat
    'MULTIPLE NECK INJURY': 'Neck and Throat', 'LARYNX': 'Neck and Throat', 'TRACHEA': 'Neck and Throat',

    # Upper and Lower Back
    'UPPER BACK AREA': 'Upper Back',
    'LOWER BACK AREA': 'Lower Back', 'LUMBAR & OR SACRAL VERTEBRAE (VERTEBRA': 'Lower Back',
    'SACRUM AND COCCYX': 'Lower Back',

    # Spinal Column
    'SPINAL CORD': 'Spinal Column', 'VERTEBRAE': 'Spinal Column',

    # Shoulder
    'SHOULDER(S)': 'Shoulder', 

    # Arm and Wrist
    'LOWER ARM': 'Arm and Wrist', 'UPPER ARM': 'Arm and Wrist', 'WRIST': 'Arm and Wrist',

    # Hand and Fingers
    'HAND': 'Hand and Fingers', 'FINGER(S)': 'Hand and Fingers', 'THUMB': 'Hand and Fingers',

    # Hip
    'HIP': 'Hip', 

    # Leg
    'LOWER LEG': 'Leg', 'UPPER LEG': 'Leg',

    # Knee
    'KNEE': 'Knee',

    # Ankle and Foot
    'ANKLE': 'Ankle and Foot', 'FOOT': 'Ankle and Foot', 'TOES': 'Ankle and Foot', 'GREAT TOE': 'Ankle and Foot',

    # Chest
    'CHEST': 'Chest',

    # Abdomen and Pelvis
    'ABDOMEN INCLUDING GROIN': 'Abdomen and Pelvis', 'PELVIS': 'Abdomen and Pelvis', 'BUTTOCKS': 'Abdomen and Pelvis',

    # Lungs
    'LUNGS': 'Lungs',

    # Heart
    'HEART': 'Heart',

    # Internal Organs
    'INTERNAL ORGANS': 'Internal Organs',

    # Body Systems
    'BODY SYSTEMS AND MULTIPLE BODY SYSTEMS': 'Body Systems', 
    'WHOLE BODY': 'Body Systems', 'MULTIPLE BODY PARTS (INCLUDING BODY': 'Body Systems',
    'ARTIFICIAL APPLIANCE': 'Body Systems',

    # Soft Tissue
    'SOFT TISSUE': 'Soft Tissue',

    # Unclassified or Insufficient Info
    'NO PHYSICAL INJURY': 'Unclassified or Insufficient Info',
    'INSUFFICIENT INFO TO PROPERLY IDENTIFY - UNCLASSIFIED': 'Unclassified or Insufficient Info'
}

### New Features Creation

Since we can't use the date variables for the models, we want to extract all the information they contain. Some variables might be exaclty the same "Accident_Year" and "Assembly_Year". However, we will calaculate all of them and exclude irrelevant ones in the feature selection part.

- __Lag_Time__: Difference in days between accident date and assembly date
- __Accident_to_C2_Lag__: Difference in days between accident date and C-2 date
- __Accident_Year__: Extract the year from Accident Date
- __Accident_Month__: Extract the month from Accident Date
- __Accident_Month_Sin__: Sine of the month from Accident Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __Accident_Month_Cos__: Cosine of the month from Accident Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __Accident_Day__: Extract the day from Accident Date
- __Reference_Days_Accident__: Starting with the first accident day and counting onwards to get a numerical order of the dates
- __Assembly_Year__: Extract the year from Assembly Date
- __Assembly_Month__: Extract the month from Assembly Date
- __Assembly_Month_Sin__: Sine of the month from Assembly Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __Assembly_Month_Cos__: Cosine of the month from Assembly Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __Assembly_Day__: Extract the day from Assembly Date
- __Reference_Days_Assembly__: Starting with the first assembly day and counting onwards to get a numerical order of the dates
- __C-2_Year__: Extract the year from C-2 Date
- __C-2_Month__: Extract the month from C-2 Date
- __C-2_Month_Sin__: Sine of the month from C-2 Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __C-2_Month_Cos__: Cosine of the month from C-2 Date; this transformation is used to represent the cyclic nature of months (ensures that December (month 12) is closer to January (month 1) in feature space)
- __C-2_Day__: Extract the day from C-2 Date
- __Reference_Days_C-2__: Starting with the first C-2 day and counting onwards to get a numerical order of the dates

In [19]:
train_df_copy = train_df.copy()

In [20]:
# Convert columns to datetime format and handle errors

train_df_copy['Accident Date'] = pd.to_datetime(train_df_copy['Accident Date'], errors='coerce')
train_df_copy['Assembly Date'] = pd.to_datetime(train_df_copy['Assembly Date'], errors='coerce')
train_df_copy['C-2 Date'] = pd.to_datetime(train_df_copy['C-2 Date'], errors='coerce')

# Check for any correlation between date columns by converting them to numeric values (days since start)
train_df_copy['Lag_Time'] = (train_df_copy['Assembly Date'] - train_df_copy['Accident Date'].min()).dt.days
train_df_copy['Accident_to_C2_Lag'] = (train_df_copy['C-2 Date'] - train_df_copy['Accident Date'].min()).dt.days


In [21]:
print(len(train_df_copy[train_df_copy['Accident Date'] > '2022-12-31']))
print(len(train_df_copy[train_df_copy['Assembly Date'] > '2022-12-31']))
print(len(train_df_copy[train_df_copy['C-2 Date'] > '2022-12-31']))
print(len(train_df_copy[train_df_copy['Accident Date'] < '2000-01-01']))


56
0
2212
1011


To keep the integrity of the dataset, it doesn't make sense to keep Accident Dates that occur after 2022 , because the assembly is only made between 2020 and 2022.  Moreover, to filter some of the noise, we will not consider accidents that have 20 years delay, because that's not very frequent and can add noise in our model.

Regarding C-2 Dates , delayed reporting by employers into 2023 is plausible, especially for claims assembled in late 2022. However we are only considering 1 year delay regarding the claims submitted in the end of 2022. 

In [22]:
outliers_date = train_df_copy[
    (train_df_copy['C-2 Date'] > '2023-12-31') | 
    (train_df_copy['Accident Date'] < '2000-01-01') |
    (train_df_copy['Accident Date'] > '2022-12-31')
]

print(len(outliers_date))

1141


In [23]:
train_df_copy = train_df_copy[
    (train_df_copy['C-2 Date'] <= '2023-12-31') & 
    (train_df_copy['Accident Date'] >= '2000-01-01') & 
    (train_df_copy['Accident Date'] <= '2022-12-31')
]

In [24]:
date_columns = ['Accident Date', 'Assembly Date', 'C-2 Date']

# Extract specific components of each date
for col in date_columns:
    train_df_copy[f'{col}_Year'] = train_df_copy[col].dt.year
    train_df_copy[f'{col}_Month'] = train_df_copy[col].dt.month
    train_df_copy[f'{col}_Day'] = train_df_copy[col].dt.dayofweek

In [25]:
train_df_copy['Accident_Year'] = train_df_copy['Accident Date_Year']
train_df_copy['Accident_Month'] = train_df_copy['Accident Date_Month']
train_df_copy['Accident_Month_Sin'] = np.sin(2 * np.pi * train_df_copy['Accident_Month'] / 12)
train_df_copy['Accident_Month_Cos'] = np.cos(2 * np.pi * train_df_copy['Accident_Month'] / 12)
train_df_copy['Accident_Day'] = train_df_copy['Accident Date_Day']

train_df_copy['Assembly_Year'] = train_df_copy['Assembly Date_Year']
train_df_copy['Assembly_Month'] = train_df_copy['Assembly Date_Month']
train_df_copy['Assembly_Month_Sin'] = np.sin(2 * np.pi * train_df_copy['Assembly_Month'] / 12)
train_df_copy['Assembly_Month_Cos'] = np.cos(2 * np.pi * train_df_copy['Assembly_Month'] / 12)
train_df_copy['Assembly_Day'] = train_df_copy['Assembly Date_Day']

train_df_copy['C-2_Year'] = train_df_copy['C-2 Date_Year']
train_df_copy['C-2_Month'] = train_df_copy['C-2 Date_Month']
train_df_copy['C-2_Month_Sin'] = np.sin(2 * np.pi * train_df_copy['C-2_Month'] / 12)
train_df_copy['C-2_Month_Cos'] = np.cos(2 * np.pi * train_df_copy['C-2_Month'] / 12)
train_df_copy['C-2_Day'] = train_df_copy['C-2 Date_Day']

In [26]:
print('First Accident Day: {}'. format(train_df_copy['Accident Date'].min()))
print('First Assembly Day: {}'. format(train_df_copy['Assembly Date'].min()))
print('First C-2 Day: {}'. format(train_df_copy['C-2 Date'].max()))

First Accident Day: 2000-01-01 00:00:00
First Assembly Day: 2020-01-01 00:00:00
First C-2 Day: 2023-12-27 00:00:00


In [27]:
print(train_df_copy['Assembly Date'].min(), train_df_copy['Assembly Date'].max())


2020-01-01 00:00:00 2022-12-31 00:00:00


In [28]:
test_df_copy = test_df.copy()

In [29]:
# Convert columns to datetime
test_df_copy['Accident Date'] = pd.to_datetime(test_df_copy['Accident Date'], errors='coerce')
test_df_copy['Assembly Date'] = pd.to_datetime(test_df_copy['Assembly Date'], errors='coerce')
test_df_copy['C-2 Date'] = pd.to_datetime(test_df_copy['C-2 Date'], errors='coerce')

# Use the reference date from train_df_copy
reference_date = train_df_copy['Accident Date'].min()

# Calculate lag features
test_df_copy['Lag_Time'] = (test_df_copy['Assembly Date'] - reference_date).dt.days
test_df_copy['Accident_to_C2_Lag'] = (test_df_copy['C-2 Date'] - reference_date).dt.days


In [30]:
outliers_date_test = test_df_copy[
    (test_df_copy['C-2 Date'] > '2023-12-31') | 
    (test_df_copy['Accident Date'] < '2000-01-01') |
    (test_df_copy['Accident Date'] > '2022-12-31')
]

print((outliers_date_test))

# Here we can't apply the same filter because we have lots of new observations that we want to predict with compensation
# they should receive  

       Accident Date  Age at Injury Alternative Dispute Resolution  \
34        2023-01-01             22                              N   
70        2023-01-01             27                              N   
104       2023-01-02             28                              N   
254       2023-01-01             74                              N   
264       2023-01-03             42                              N   
...              ...            ...                            ...   
387968    2024-05-13             38                              N   
387969    2024-05-15             58                              N   
387971    2024-05-22             59                              N   
387972    2024-05-06             45                              N   
387973    2024-02-24             42                              N   

       Assembly Date Attorney/Representative  Average Weekly Wage  Birth Year  \
34        2023-01-02                       N                  0.0         0.0 

In [31]:
date_columns = ['Accident Date', 'Assembly Date', 'C-2 Date']

# Extract specific components of each date
for col in date_columns:
    test_df_copy[f'{col}_Year'] = test_df_copy[col].dt.year
    test_df_copy[f'{col}_Month'] = test_df_copy[col].dt.month
    test_df_copy[f'{col}_Day'] = test_df_copy[col].dt.dayofweek

In [32]:
test_df_copy['Accident_Year'] = test_df_copy['Accident Date_Year']
test_df_copy['Accident_Month'] = test_df_copy['Accident Date_Month']
test_df_copy['Accident_Month_Sin'] = np.sin(2 * np.pi * test_df_copy['Accident_Month'] / 12)
test_df_copy['Accident_Month_Cos'] = np.cos(2 * np.pi * test_df_copy['Accident_Month'] / 12)
test_df_copy['Accident_Day'] = test_df_copy['Accident Date_Day']

test_df_copy['Assembly_Year'] = test_df_copy['Assembly Date_Year']
test_df_copy['Assembly_Month'] = test_df_copy['Assembly Date_Month']
test_df_copy['Assembly_Month_Sin'] = np.sin(2 * np.pi * test_df_copy['Assembly_Month'] / 12)
test_df_copy['Assembly_Month_Cos'] = np.cos(2 * np.pi * test_df_copy['Assembly_Month'] / 12)
test_df_copy['Assembly_Day'] = test_df_copy['Assembly Date_Day']

test_df_copy['C-2_Year'] = test_df_copy['C-2 Date_Year']
test_df_copy['C-2_Month'] = test_df_copy['C-2 Date_Month']
test_df_copy['C-2_Month_Sin'] = np.sin(2 * np.pi * test_df_copy['C-2_Month'] / 12)
test_df_copy['C-2_Month_Cos'] = np.cos(2 * np.pi * test_df_copy['C-2_Month'] / 12)
test_df_copy['C-2_Day'] = test_df_copy['C-2 Date_Day']

In [33]:
# drop dateformat variables because they can't be used
train_df_copy.drop(columns=['Accident Date', 'Assembly Date', 'C-2 Date'], inplace=True)

In [34]:
# drop dateformat variables because they can't be used
test_df_copy.drop(columns=['Accident Date', 'Assembly Date', 'C-2 Date'], inplace=True)

In [35]:
train_df_copy['Cause Injury Category'] = train_df_copy['WCIO Cause of Injury Description'].map(injury_mapping).fillna('Other')

test_df_copy['Cause Injury Category'] = test_df_copy['WCIO Cause of Injury Description'].map(injury_mapping).fillna('Other')

# drop the original variable and only keep the groups
train_df_copy.drop(columns='WCIO Cause of Injury Description', inplace=True)

test_df_copy.drop(columns='WCIO Cause of Injury Description', inplace=True)

In [36]:
train_df_copy['Nature of Injury Category'] = train_df_copy['WCIO Nature of Injury Description'].map(injury_nature_mapping).fillna('Other')

test_df_copy['Nature of Injury Category'] = test_df_copy['WCIO Nature of Injury Description'].map(injury_nature_mapping).fillna('Other')

train_df_copy.drop(columns='WCIO Nature of Injury Description', inplace=True)

test_df_copy.drop(columns='WCIO Nature of Injury Description', inplace=True)

In [37]:
train_df_copy['Body Part Category'] = train_df_copy['WCIO Part Of Body Description'].map(body_part_mapping).fillna('Other')

test_df_copy['Body Part Category'] = test_df_copy['WCIO Part Of Body Description'].map(body_part_mapping).fillna('Other')


train_df_copy.drop(columns='WCIO Part Of Body Description', inplace=True)

test_df_copy.drop(columns='WCIO Part Of Body Description', inplace=True)

In [38]:
# Gender: Combine "U" and "X" into "Other"
train_df_copy['Gender'] = train_df_copy['Gender'].replace({'U': 'Other', 'X': 'Other'})

test_df_copy['Gender'] = test_df_copy['Gender'].replace({'U': 'Other', 'X': 'Other'})


In [39]:
# Define age bins and labels
age_bins = [0, 19, 29, 39, 49, 59, float('inf')]
age_labels = ['Under 20', '20-29', '30-39', '40-49', '50-59', '60 and above']

# Create the age groups for train and test datasets
train_df_copy['Age_Group'] = pd.cut(train_df_copy['Age at Injury'], bins=age_bins, labels=age_labels)
test_df_copy['Age_Group'] = pd.cut(test_df_copy['Age at Injury'], bins=age_bins, labels=age_labels)

In [78]:
# Define wage bins and labels
wage_bins = [-np.inf, 1, 700, 1100, 1500, max(train_df_copy['Average Weekly Wage']) + 1]  # Use -np.inf to include 0.00
wage_labels = ['Zero', 'Below Average', 'Average', 'Above Average', 'High']

# Create the wage groups for train and test datasets
train_df_copy['Wage_Group'] = pd.cut(train_df_copy['Average Weekly Wage'], bins=wage_bins, labels=wage_labels)
test_df_copy['Wage_Group'] = pd.cut(test_df_copy['Average Weekly Wage'], bins=wage_bins, labels=wage_labels)

In [41]:
carrier_counts = train_df_copy['Carrier Name'].value_counts()
carrier_counts

Carrier Name
STATE INSURANCE FUND              108995
POLICE, FIRE, SANITATION           20552
AMERICAN ZURICH INSURANCE CO       17140
CHARTER OAK FIRE INS CO            16817
INDEMNITY INS. OF N AMERICA        14099
                                   ...  
MARATHON CENTRAL SCHOOL DIST           1
SAFETY FIRST INSURANCE COMPANY         1
DELHI CENTRAL SCHOOL DISTRICT          1
CAMBRIDGE CENTRAL SCHOOL               1
HERMON-DEKALB CENTRAL                  1
Name: count, Length: 1996, dtype: int64

In [42]:
treshold = 5000

train_df_copy['Carrier Name Grouped'] = train_df_copy['Carrier Name'].apply(
    lambda x: x if carrier_counts.get(x, 0) > treshold else 'Other'
)

test_df_copy['Carrier Name Grouped'] = test_df_copy['Carrier Name'].apply(
    lambda x: x if carrier_counts.get(x, 0) > treshold else 'Other'
)

In [43]:
train_df_copy['WCB Decision'].unique();
train_df_copy['WCB Decision'].value_counts() # 1 value only so we should drop it

WCB Decision
Not Work Related    556330
Name: count, dtype: int64

In [44]:
train_df_copy.drop(columns='WCB Decision', inplace=True)

In [45]:
train_df_copy.columns

Index(['Age at Injury', 'Alternative Dispute Resolution',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'C-3 Date', 'Carrier Name', 'Carrier Type', 'Claim Identifier',
       'Claim Injury Type', 'County of Injury', 'COVID-19 Indicator',
       'District Name', 'First Hearing Date', 'Gender', 'IME-4 Count',
       'Industry Code', 'Industry Code Description', 'Medical Fee Region',
       'OIICS Nature of Injury Description', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Zip Code',
       'Agreement Reached', 'Number of Dependents', 'Lag_Time',
       'Accident_to_C2_Lag', 'Accident Date_Year', 'Accident Date_Month',
       'Accident Date_Day', 'Assembly Date_Year', 'Assembly Date_Month',
       'Assembly Date_Day', 'C-2 Date_Year', 'C-2 Date_Month', 'C-2 Date_Day',
       'Accident_Year', 'Accident_Month', 'Accident_Month_Sin',
       'Accident_Month_Cos', 'Accident_Day', 'Assembly_Year', 'Assembly_Month',
 

In [46]:
test_df_copy.columns

Index(['Age at Injury', 'Alternative Dispute Resolution',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'C-3 Date', 'Carrier Name', 'Carrier Type', 'Claim Identifier',
       'County of Injury', 'COVID-19 Indicator', 'District Name',
       'First Hearing Date', 'Gender', 'IME-4 Count', 'Industry Code',
       'Industry Code Description', 'Medical Fee Region',
       'OIICS Nature of Injury Description', 'WCIO Cause of Injury Code',
       'WCIO Nature of Injury Code', 'WCIO Part Of Body Code', 'Zip Code',
       'Number of Dependents', 'Lag_Time', 'Accident_to_C2_Lag',
       'Accident Date_Year', 'Accident Date_Month', 'Accident Date_Day',
       'Assembly Date_Year', 'Assembly Date_Month', 'Assembly Date_Day',
       'C-2 Date_Year', 'C-2 Date_Month', 'C-2 Date_Day', 'Accident_Year',
       'Accident_Month', 'Accident_Month_Sin', 'Accident_Month_Cos',
       'Accident_Day', 'Assembly_Year', 'Assembly_Month', 'Assembly_Month_Sin',
       'Assembly_Mont

In [47]:
for column in train_df_copy.columns:
    if column not in test_df_copy.columns:
        print(column)

# The columns Agreement Reached is not present in the test_df dataset so we should drop it

Claim Injury Type
Agreement Reached


In [48]:
train_df_copy['Agreement Reached'].unique();
train_df_copy['Agreement Reached'].value_counts() 

Agreement Reached
0.0    530536
1.0     25794
Name: count, dtype: int64

In [49]:
train_df_copy.drop(columns='Agreement Reached', inplace=True)

In [50]:
train_df_copy.drop(columns='OIICS Nature of Injury Description', inplace=True)
test_df_copy.drop(columns='OIICS Nature of Injury Description', inplace=True)

# Missing Values Handling 

In [51]:
train_df.isnull().sum()

Accident Date                          23134
Age at Injury                          19445
Alternative Dispute Resolution         19445
Assembly Date                              0
Attorney/Representative                19445
Average Weekly Wage                    48096
Birth Year                             48523
C-2 Date                               34005
C-3 Date                              406226
Carrier Name                           19445
Carrier Type                           19445
Claim Identifier                           0
Claim Injury Type                      19445
County of Injury                       19445
COVID-19 Indicator                     19445
District Name                          19445
First Hearing Date                    442673
Gender                                 19445
IME-4 Count                           460668
Industry Code                          29403
Industry Code Description              29403
Medical Fee Region                     19445
OIICS Natu

In [137]:
numeric_cols_train = train_df_copy.select_dtypes(include=np.number).columns.tolist()

categorical_cols_train = train_df_copy.select_dtypes('object').columns.tolist()

In [125]:
numeric_cols_test = [col for col in numeric_cols if col in test_df_copy.columns]

categorical_cols_test = [col for col in numeric_cols if col in test_df_copy.columns]

In [ ]:
train_df_copy[categorical_cols_train].columns

Index(['Attorney/Representative', 'Carrier Name', 'Carrier Type',
       'Claim Injury Type', 'County of Injury', 'COVID-19 Indicator',
       'District Name', 'Gender', 'Industry Code Description',
       'Medical Fee Region', 'Zip Code', 'Cause Injury Category',
       'Nature of Injury Category', 'Body Part Category',
       'Carrier Name Grouped'],
      dtype='object')

In [102]:
train_df_copy.drop(columns='Alternative Dispute Resolution', inplace=True)  
test_df_copy.drop(columns='Alternative Dispute Resolution', inplace=True)

# Handling NaNs for numerical features training set

In [ ]:
train_df_copy[numeric_cols_train].isnull().sum()

Age at Injury                    0
Average Weekly Wage              0
Claim Identifier                 0
Industry Code                 1390
WCIO Cause of Injury Code      941
WCIO Nature of Injury Code     953
WCIO Part Of Body Code        2509
Number of Dependents             0
Lag_Time                         0
Accident_to_C2_Lag               0
Accident Date_Year               0
Accident Date_Month              0
Accident Date_Day                0
Assembly Date_Year               0
Assembly Date_Month              0
Assembly Date_Day                0
C-2 Date_Year                    0
C-2 Date_Month                   0
C-2 Date_Day                     0
Accident_Year                    0
Accident_Month                   0
Accident_Month_Sin               0
Accident_Month_Cos               0
Accident_Day                     0
Assembly_Year                    0
Assembly_Month                   0
Assembly_Month_Sin               0
Assembly_Month_Cos               0
Assembly_Day        

In [135]:
for column in ["Industry Code", "WCIO Cause of Injury Code", "WCIO Nature of Injury Code", "WCIO Part Of Body Code"]:
    mode_value = test_df_copy[column].mode()[0]
    test_df_copy[column].fillna(mode_value, inplace=True)

In [126]:
non_zero_median_wage_train = train_df_copy[train_df_copy["Average Weekly Wage"] > 0]["Average Weekly Wage"].median()
print(f"Median of Non-Zero Wages: {non_zero_median_wage_train}")

Median of Non-Zero Wages: 1068.76


In [ ]:
train_df_copy["Average Weekly Wage"].fillna(non_zero_median_wage_train, inplace=True)

In [138]:
train_df_copy[numeric_cols_train].isnull().sum()

Age at Injury                 0
Average Weekly Wage           0
Claim Identifier              0
Industry Code                 0
WCIO Cause of Injury Code     0
WCIO Nature of Injury Code    0
WCIO Part Of Body Code        0
Number of Dependents          0
Lag_Time                      0
Accident_to_C2_Lag            0
Accident Date_Year            0
Accident Date_Month           0
Accident Date_Day             0
Assembly Date_Year            0
Assembly Date_Month           0
Assembly Date_Day             0
C-2 Date_Year                 0
C-2 Date_Month                0
C-2 Date_Day                  0
Accident_Year                 0
Accident_Month                0
Accident_Month_Sin            0
Accident_Month_Cos            0
Accident_Day                  0
Assembly_Year                 0
Assembly_Month                0
Assembly_Month_Sin            0
Assembly_Month_Cos            0
Assembly_Day                  0
C-2_Year                      0
C-2_Month                     0
C-2_Mont

# Handling NaNs for numerical features test set

In [116]:
test_df_copy[numeric_cols_test].isnull().sum()

Age at Injury                     0
Average Weekly Wage           19204
Claim Identifier                  0
Industry Code                  7736
WCIO Cause of Injury Code     10348
WCIO Nature of Injury Code    10560
WCIO Part Of Body Code         9549
Number of Dependents              0
Lag_Time                          0
Accident_to_C2_Lag             9134
Accident Date_Year             2444
Accident Date_Month            2444
Accident Date_Day              2444
Assembly Date_Year                0
Assembly Date_Month               0
Assembly Date_Day                 0
C-2 Date_Year                  9134
C-2 Date_Month                 9134
C-2 Date_Day                   9134
Accident_Year                  2444
Accident_Month                 2444
Accident_Month_Sin             2444
Accident_Month_Cos             2444
Accident_Day                   2444
Assembly_Year                     0
Assembly_Month                    0
Assembly_Month_Sin                0
Assembly_Month_Cos          

In [122]:
non_zero_median_wage_test = test_df_copy[test_df_copy["Average Weekly Wage"] > 0]["Average Weekly Wage"].median()
print(f"Median of Non-Zero Wages: {non_zero_median_wage_test}")

Median of Non-Zero Wages: 1048.255


In [127]:
test_df_copy["Average Weekly Wage"].fillna(non_zero_median_wage_test, inplace=True)

In [129]:
test_df_copy[numeric_cols_test].isnull().sum()

Age at Injury                    0
Average Weekly Wage              0
Claim Identifier                 0
Industry Code                    0
WCIO Cause of Injury Code        0
WCIO Nature of Injury Code       0
WCIO Part Of Body Code           0
Number of Dependents             0
Lag_Time                         0
Accident_to_C2_Lag            9134
Accident Date_Year            2444
Accident Date_Month           2444
Accident Date_Day             2444
Assembly Date_Year               0
Assembly Date_Month              0
Assembly Date_Day                0
C-2 Date_Year                 9134
C-2 Date_Month                9134
C-2 Date_Day                  9134
Accident_Year                 2444
Accident_Month                2444
Accident_Month_Sin            2444
Accident_Month_Cos            2444
Accident_Day                  2444
Assembly_Year                    0
Assembly_Month                   0
Assembly_Month_Sin               0
Assembly_Month_Cos               0
Assembly_Day        

In [133]:
for column in ["Accident_to_C2_Lag",
    "Accident_Year",
    "Accident Date_Month",
    "Accident Date_Day",
    "Accident_Month",
    "Accident_Month_Sin",
    "Accident_Month_Cos",
    "Accident_Day",
    "C-2 Date_Year",
    "C-2 Date_Month",
    "C-2 Date_Day",
    "C-2_Month_Sin",
    "C-2_Month_Cos",
    "C-2_Day"]:
    median_value = test_df_copy[column].median()
    test_df_copy[column].fillna(mode_value, inplace=True)

In [134]:
test_df_copy[numeric_cols_test].isnull().sum()

Age at Injury                 0
Average Weekly Wage           0
Claim Identifier              0
Industry Code                 0
WCIO Cause of Injury Code     0
WCIO Nature of Injury Code    0
WCIO Part Of Body Code        0
Number of Dependents          0
Lag_Time                      0
Accident_to_C2_Lag            0
Accident Date_Year            0
Accident Date_Month           0
Accident Date_Day             0
Assembly Date_Year            0
Assembly Date_Month           0
Assembly Date_Day             0
C-2 Date_Year                 0
C-2 Date_Month                0
C-2 Date_Day                  0
Accident_Year                 0
Accident_Month                0
Accident_Month_Sin            0
Accident_Month_Cos            0
Accident_Day                  0
Assembly_Year                 0
Assembly_Month                0
Assembly_Month_Sin            0
Assembly_Month_Cos            0
Assembly_Day                  0
C-2_Year                      0
C-2_Month                     0
C-2_Mont

# Handling NaNs for categorical features training set

In [139]:
train_df_copy[categorical_cols_train].isnull().sum()

Attorney/Representative          0
Carrier Name                     0
Carrier Type                     0
Claim Injury Type                0
County of Injury                 0
COVID-19 Indicator               0
District Name                    0
Gender                           0
Industry Code Description     1390
Medical Fee Region               0
Zip Code                     27755
Cause Injury Category            0
Nature of Injury Category        0
Body Part Category               0
Carrier Name Grouped             0
dtype: int64

In [140]:
for column in ['Zip Code','Industry Code Description']:
    mode_value = train_df_copy[column].mode()[0]  # Calculate the mode
    train_df_copy[column].fillna(mode_value, inplace=True)  # Replace missing values with mode

As we have low amount of missing values for Industry Code(1390) , WCIO Cause of Injury Code(941), WCIO Nature of Injury Code(953) and WCIO Part Of Body Code(2509) , (<1%) of missing values compared to the length of the dataset, the mode (most frequent value) is a simple and effective choice

In [113]:
for column in ["Industry Code", "WCIO Cause of Injury Code", "WCIO Nature of Injury Code", "WCIO Part Of Body Code"]:
    mode_value = train_df_copy[column].mode()[0]
    train_df_copy[column].fillna(mode_value, inplace=True)


Let's have a look at the three variables with the high percentage of missing values:
- __IME-4 Count (77.62%)__: This variable describes the number of IME-4 forms received per claim. The missing values could be replaced with zeros, however, since the amout of missing values is so high, it's variance is relatively small.
- __First Hearing Date (74.59%)__: The description of the variable states that a blank date means the claim has not yet had a hearing held. The relationship between the date variables should be checked to determine if its relevant. Further, if the variable has an impact on the target variable, one idea would be to transform it into a binary variable which states if the first hearing date took place already (=1) or not (=0).
- __C-3 Date (68.45%)__: This variable means that the Date Form C-3 (Employee Claim Form) was received. It could also be transformed into a binary variable if it is important.

In [63]:
train_df_copy.drop(columns='IME-4 Count',inplace=True)
test_df_copy.drop(columns='IME-4 Count',inplace=True)

In [141]:
train_df_copy[categorical_cols_train].isnull().sum()

Attorney/Representative      0
Carrier Name                 0
Carrier Type                 0
Claim Injury Type            0
County of Injury             0
COVID-19 Indicator           0
District Name                0
Gender                       0
Industry Code Description    0
Medical Fee Region           0
Zip Code                     0
Cause Injury Category        0
Nature of Injury Category    0
Body Part Category           0
Carrier Name Grouped         0
dtype: int64

## Handling NaNs for categorical features

In [143]:
test_df_copy[categorical_cols_test].isnull().sum()

Age at Injury                 0
Average Weekly Wage           0
Claim Identifier              0
Industry Code                 0
WCIO Cause of Injury Code     0
WCIO Nature of Injury Code    0
WCIO Part Of Body Code        0
Number of Dependents          0
Lag_Time                      0
Accident_to_C2_Lag            0
Accident Date_Year            0
Accident Date_Month           0
Accident Date_Day             0
Assembly Date_Year            0
Assembly Date_Month           0
Assembly Date_Day             0
C-2 Date_Year                 0
C-2 Date_Month                0
C-2 Date_Day                  0
Accident_Year                 0
Accident_Month                0
Accident_Month_Sin            0
Accident_Month_Cos            0
Accident_Day                  0
Assembly_Year                 0
Assembly_Month                0
Assembly_Month_Sin            0
Assembly_Month_Cos            0
Assembly_Day                  0
C-2_Year                      0
C-2_Month                     0
C-2_Mont

In [ ]:
train_df_copy.drop(columns='First Hearing Date',inplace=True)
test_df_copy.drop(columns='First Hearing Date',inplace=True)

In [71]:
train_df_copy.drop(columns='C-3 Date',inplace=True)
test_df_copy.drop(columns='C-3 Date',inplace=True)

In [108]:
train_df_copy['Claim Injury Type'].value_counts()   

Claim Injury Type
2. NON-COMP        281849
4. TEMPORARY       147372
3. MED ONLY         68620
5. PPD SCH LOSS     48242
1. CANCELLED         5470
6. PPD NSL           4211
8. DEATH              470
7. PTD                 96
Name: count, dtype: int64

In [ ]:
train_df_copy["Claim Type Numeric"] = train_df_copy["Claim Injury Type"].astype("category").cat.codes

In [ ]:
# Checking the correlations
correlations = {
    "Age at Injury vs Claim Type Numeric": train_df_copy["Age at Injury"].corr(train_df_copy["Claim Type Numeric"],method='spearman'),
    "Birth Year vs Claim Type Numeric": train_df_copy["Birth Year"].corr(train_df_copy["Claim Type Numeric"],method='spearman'),
    "Age at Injury vs Birth Year": train_df_copy["Age at Injury"].corr(train_df_copy["Birth Year"],method='spearman'),
}

correlations

In [ ]:
train_df_copy.drop(columns='Birth Year',inplace=True)
test_df_copy.drop(columns='Birth Year',inplace=True)

# Saving the pre-processing

In [ ]:
import os

In [144]:
# Define the absolute path for saving files
processed_dir = "C:/Users/alexg/Ambiente de Trabalho/Mestrado Data Science NOVA IMS/1st semester/Machine Learning/Project_github/ML_Project_Group52/project/data/processed"

In [145]:
# Ensure the directory exists
os.makedirs(processed_dir, exist_ok=True)

# Define the full file paths
train_file_path = os.path.join(processed_dir, 'prepared_claims_data_train.csv')
test_file_path = os.path.join(processed_dir, 'prepared_claims_data_test.csv')

# Save the datasets
train_df_copy.to_csv(train_file_path, index=False)
test_df_copy.to_csv(test_file_path, index=False)

print(f"Training data saved to: {train_file_path}")
print(f"Test data saved to: {test_file_path}")


Training data saved to: C:/Users/alexg/Ambiente de Trabalho/Mestrado Data Science NOVA IMS/1st semester/Machine Learning/Project_github/ML_Project_Group52/project/data/processed\prepared_claims_data_train.csv
Test data saved to: C:/Users/alexg/Ambiente de Trabalho/Mestrado Data Science NOVA IMS/1st semester/Machine Learning/Project_github/ML_Project_Group52/project/data/processed\prepared_claims_data_test.csv
